# Part 1. Install and Confirgure Ollama and all necessary packages


First, I will install all the requirement packages

In [95]:
# Install all the requirement packages
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh # Install Ollama v0.1.30

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10941    0 10941    0     0  10717      0 --:--:--  0:00:01 --:--:-- 10726>>> Downloading ollama...
100 10941    0 10941    0     0  10390      0 --:--:--  0:00:01 --:--:-- 10400
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [69]:
# Install Prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb
!pip install sentence-transformers
!pip install llama-index
!pip install requests
!pip install transformers

In [62]:
import os
import openai
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline, BertTokenizer, BertModel

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document, QueryBundle
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


from llama_index.core import Settings, StorageContext
from llama_index.llms.ollama import Ollama

from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core.node_parser import SentenceWindowNodeParser, SentenceSplitter
from transformers import pipeline, BertTokenizer, BertModel

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
import subprocess
import time

# Develop a prompt template
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

from sklearn.metrics.pairwise import cosine_similarity

# Import required modules from llama_index library for HyDE and Multi-Step Transformations
from llama_index.core.indices.query.query_transform.base import HyDEQueryTransform, StepDecomposeQueryTransform
from llama_index.core.query_engine import TransformQueryEngine, MultiStepQueryEngine

# Import required package for GPT2
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

Next, I will set up Ollama Model

In [97]:
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # print the global variable to check it saved

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Process ID:", process.pid) # print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check


phi:latest
Process ID: 362484
ollama version is 0.1.41


Now that everything should be set up, we can query the model and get it to generate some text about Tesla and Elon Musk:

In [5]:
# Query the model via the command line to ensure it works
!ollama run $OLLAMA_MODEL "Give me short definition of Tesla and Elon Musk."

⠙ ⠹ ⠸ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹  Tesla is an electric vehicle and clean energy company founded by entrepreneur and inventor, Elon Musk. It produces high-performance electric cars, batteries for electric vehicles, solar panels, and other related products. Elon Musk is a cofounder and CEO of the company. He is also known for his work on SpaceX, which aims to make space travel more accessible and sustainable.




I will check the response of Llama Index model (Ollama model) for my set of tests/ queries before building RAG system

In [76]:
# Query the model via the command line to ensure it works
!ollama run $OLLAMA_MODEL "How much did Twitter's stock surge after Elon Musk disclosed his stake in the company? "

⠙ ⠙ ⠹ ⠸ ⠼ ⠴ ⠧ ⠇ ⠏ ⠋ ⠙ ⠙ ⠹ ⠸ ⠼ ⠴ ⠧ ⠧ ⠏ ⠋ ⠙ ⠙  As an AI language model, I don't have access to real-time financial data or market trends. However, according to news reports, Tesla's $1 billion investment in Twitter and a $2.5 billion debt-to-equity swap deal announced by Musk led to a significant increase in the company's stock price. The value of Elon Musk's stake in Twitter also increased considerably after the transaction was completed.




In [77]:
!ollama run $OLLAMA_MODEL "What percentage of Musk's followers agreed that Twitter's algorithm should be open-source? "

⠙  I do not have access to real-time information or statistics, so i cannot provide you with up-to-date data on this topic. however, according to an article by techcrunch in 2020, only a small percentage of musk's followers agreed that twitters algorithm should be open-source.




In [78]:
!ollama run $OLLAMA_MODEL "How did the shares of larger lithium mining companies, Albemarle and SQM, react to Musk's tweet lamenting the rising cost of lithium? "

⠙  According to news reports, both Albemarle and SQM saw their stock prices fall in response to Elon Musk's tweet about the rising cost of lithium. However, they quickly recovered as investors realized that Musk's concerns were not entirely new and had been raised by other experts in the industry.




In [79]:
!ollama run $OLLAMA_MODEL "What were the claims of the California Department of Fair Employment and Housing against Tesla? "

⠙  According to the article, the California Department of Fair Employment and Housing (DFEH) filed a lawsuit against Tesla for workplace discrimination and harassment allegations made by several employees. The DFEH claims that Tesla's policies and practices have created a hostile work environment, resulting in low employee morale, high turnover, and decreased productivity. Specifically, the lawsuit alleges that Tesla has allowed discriminatory remarks and behavior to go unchecked, particularly towards women and minorities.




In [101]:
!ollama run $OLLAMA_MODEL "Discuss the broader implications of the Federal Reserve's rate hikes on the tech sector as articulated by both Elon Musk and Cathie Wood. "

⠙  Certainly, let me provide you with some insights into this topic.

Elon Musk has been vocal about his concerns regarding the impact of interest rates on the technology industry. In particular, he has expressed worry that higher interest rates could lead to a slowdown in investment and innovation in the sector. This is because higher borrowing costs may make it more expensive for tech companies to access capital, which could limit their ability to grow and expand their operations.

On the other hand, Cathie Wood has been a proponent of the idea that rising interest rates can be positive for the technology industry. She argues that higher interest rates can help to reduce inflationary pressures and promote economic stability, which in turn may create more favorable conditions for tech companies to thrive. Additionally, some investors believe that rising rates could lead to increased demand for tech products and services as consumers seek to manage their finances more effectively.

Ove

In [100]:
!ollama run $OLLAMA_MODEL "What is the lastest news about Apple in 2022? "

⠙  I am not able to access live information on the internet. however, as of today, apple announced that it will be releasing its first all-in-one computer with a 12.9" display and an iMac design. additionally, the company also released the iPad Pro 11 in 2021. if you are looking for more news about apple, i can help you find it on various news websites.




In [83]:
!ollama run $OLLAMA_MODEL "What is the current employment situation in the UK in 2024? "

⠙  I can tell you that the uk government, or the united kingdom government, is the administrative body responsible for making laws and policies in the United Kingdom. it's made up of various branches such as the executive branch (led by the prime minister) and the legislative branch (informally known as parliament). their main goal is to serve the people of the uk and maintain order in the country.




# Part 2: Read data & Data Preparation

## 2.1 Read data

In [111]:
# Read data
# Read the downloaded csv file on Github
Tesla_news = pd.read_csv('Tesla_news.csv', encoding='ISO-8859-1')

In [112]:
# Display the first few rows to understand the structure of the dataset
Tesla_news.head()

,no.,date,source,title,link,content
0,1,3-Jan-22,The New York Observer,"Tesla Crushed Every Expectation in 2021, Kicki...",https://www.proquest.com/usnews/docview/261613...,A Tesla Model X on display at the 3rd China In...
1,2,8-Jan-22,Barron's (Online),Tesla's New Austin Factory to Start Up Soon. I...,https://www.proquest.com/usnews/docview/261780...,Christmas arrived a little late for Tesla bull...
2,3,13-Jan-22,Wall Street Journal (Online),"Elon Musk, Hollywood Actors Criticize Californ...",https://www.proquest.com/usnews/docview/261928...,A proposal to cut incentives for rooftop solar...
3,4,14-Jan-22,Wall Street Journal (Online),Elon Musk-Backed EV Battery Technology Rises t...,https://www.proquest.com/usnews/docview/261953...,A less-expensive battery technology championed...
4,5,17-Jan-22,Republican & Herald,Smart progressives should love Elon Musk,https://www.proquest.com/usnews/docview/262085...,Nearly everything Sen. Elizabeth Warren tweete...


In [113]:
# Combine date, title, and content into a single column
Tesla_news['combined_text'] = Tesla_news.apply(lambda row: f"{row['date']} - {row['title']} - {row['content']}", axis=1)

# Display the first few rows to check the combined column
Tesla_news.head()

,no.,date,source,title,link,content,combined_text
0,1,3-Jan-22,The New York Observer,"Tesla Crushed Every Expectation in 2021, Kicki...",https://www.proquest.com/usnews/docview/261613...,A Tesla Model X on display at the 3rd China In...,3-Jan-22 - Tesla Crushed Every Expectation in ...
1,2,8-Jan-22,Barron's (Online),Tesla's New Austin Factory to Start Up Soon. I...,https://www.proquest.com/usnews/docview/261780...,Christmas arrived a little late for Tesla bull...,8-Jan-22 - Tesla's New Austin Factory to Start...
2,3,13-Jan-22,Wall Street Journal (Online),"Elon Musk, Hollywood Actors Criticize Californ...",https://www.proquest.com/usnews/docview/261928...,A proposal to cut incentives for rooftop solar...,"13-Jan-22 - Elon Musk, Hollywood Actors Critic..."
3,4,14-Jan-22,Wall Street Journal (Online),Elon Musk-Backed EV Battery Technology Rises t...,https://www.proquest.com/usnews/docview/261953...,A less-expensive battery technology championed...,14-Jan-22 - Elon Musk-Backed EV Battery Techno...
4,5,17-Jan-22,Republican & Herald,Smart progressives should love Elon Musk,https://www.proquest.com/usnews/docview/262085...,Nearly everything Sen. Elizabeth Warren tweete...,17-Jan-22 - Smart progressives should love Elo...


## 2.2 Data Preprocessing/ Data cleaning

In [114]:
# Data Cleaning and Preprocessing
# Remove duplicates
Tesla_news.drop_duplicates(inplace=True)

# Handle missing values if necessary (e.g., drop rows with missing values)
Tesla_news.dropna(inplace=True)

In [115]:
# Data Cleaning process
def clean_text(text):
    # Remove HTML-like entities
    text = re.sub(r"&[a-zA-Z]+;", ' ', text)

    # Replace special characters specific to encoding issues
    text = re.sub(r"â€”", '-', text)
    text = re.sub(r"â€“", '-', text)
    text = re.sub(r"â€¦", '...', text)
    text = re.sub(r"â€œ", '"', text)
    text = re.sub(r"â€", '"', text)
    text = re.sub(r"â€˜", "'", text)
    text = re.sub(r"â€™", "'", text)

    # Remove any remaining non-ASCII (American Standard Code for Information Interchange) characters
    text = re.sub(r"[^\x00-\x7F]+", ' ', text)

    # Remove URLs
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)

    # Remove Twitter handles or mentions
    text = re.sub(r"@[A-Za-z0-9_]+", ' ', text)

    # Replace consecutive dots with a single dot
    text = re.sub(r'\.{2,}', '.', text)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [116]:
# Apply the cleaning function to the 'text' column
Tesla_news['clean_combined_text'] = Tesla_news['combined_text'].apply(clean_text)

# Display a few examples of cleaned text
Tesla_news

,no.,date,source,title,link,content,combined_text,clean_combined_text
0,1,3-Jan-22,The New York Observer,"Tesla Crushed Every Expectation in 2021, Kicki...",https://www.proquest.com/usnews/docview/261613...,A Tesla Model X on display at the 3rd China In...,3-Jan-22 - Tesla Crushed Every Expectation in ...,3-Jan-22 - Tesla Crushed Every Expectation in ...
1,2,8-Jan-22,Barron's (Online),Tesla's New Austin Factory to Start Up Soon. I...,https://www.proquest.com/usnews/docview/261780...,Christmas arrived a little late for Tesla bull...,8-Jan-22 - Tesla's New Austin Factory to Start...,8-Jan-22 - Tesla's New Austin Factory to Start...
2,3,13-Jan-22,Wall Street Journal (Online),"Elon Musk, Hollywood Actors Criticize Californ...",https://www.proquest.com/usnews/docview/261928...,A proposal to cut incentives for rooftop solar...,"13-Jan-22 - Elon Musk, Hollywood Actors Critic...","13-Jan-22 - Elon Musk, Hollywood Actors Critic..."
3,4,14-Jan-22,Wall Street Journal (Online),Elon Musk-Backed EV Battery Technology Rises t...,https://www.proquest.com/usnews/docview/261953...,A less-expensive battery technology championed...,14-Jan-22 - Elon Musk-Backed EV Battery Techno...,14-Jan-22 - Elon Musk-Backed EV Battery Techno...
4,5,17-Jan-22,Republican & Herald,Smart progressives should love Elon Musk,https://www.proquest.com/usnews/docview/262085...,Nearly everything Sen. Elizabeth Warren tweete...,17-Jan-22 - Smart progressives should love Elo...,17-Jan-22 - Smart progressives should love Elo...
...,...,...,...,...,...,...,...,...
545,546,30-Dec-22,"Wall Street Journal, Eastern edition",Biden's Back Door Tesla Subsidy,https://www.proquest.com/usnews/docview/275916...,Tesla's stock market value has plunged by some...,30-Dec-22 - Biden's Back Door Tesla Subsidy - ...,30-Dec-22 - Biden's Back Door Tesla Subsidy - ...
546,547,30-Dec-22,"New York Times, Late Edition (East Coast)","It Was a Bad Year for Stocks, and a Worse Year...",https://www.proquest.com/usnews/docview/275936...,FILE â Tesla vehicles charge at the One Pase...,"30-Dec-22 - It Was a Bad Year for Stocks, and ...","30-Dec-22 - It Was a Bad Year for Stocks, and ..."
547,548,30-Dec-22,"Wall Street Journal, Eastern edition","Musk, Wood Are Almost Right About The Fed",https://www.proquest.com/usnews/docview/275916...,[Financial Analysis and Commentary]\nSome of t...,"30-Dec-22 - Musk, Wood Are Almost Right About ...","30-Dec-22 - Musk, Wood Are Almost Right About ..."
548,549,30-Dec-22,"New York Times, Late Edition (East Coast)",Looking Back on a Sobering Year in Silicon Val...,https://www.proquest.com/usnews/docview/275915...,A conversation with The Times's technology edi...,30-Dec-22 - Looking Back on a Sobering Year in...,30-Dec-22 - Looking Back on a Sobering Year in...


# Part 3: Building RAG system

With everything installed we can setup (in Python) our LLM model:

In [117]:
# Setup LLM model
llm = Ollama(model=OLLAMA_MODEL, request_timeout=7200)
# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm

## 3.1 Chunk and Embed the Knowledge-Base


In [118]:
# Initialize a HuggingFace Embedding model

embedding_models = {
    "BAAI/bge-small-en-v1.5": HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    "sentence-transformers/all-mpnet-base-v2": HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


I defined semantic_chunk_text, using the SemanticSplitterNodeParser with a buffer size of 1 and a breakpoint percentile threshold of 90. This function takes the text of each document, applies the embedding model, and splits the text into semantically meaningful chunks. This approach ensures that each chunk retains contextual integrity, crucial for accurate information retrieval and generation.

In [119]:
# Define the function to chunk the text semantically (Semantic chunking)
def semantic_chunk_text(text, embed_model):
    parser = SemanticSplitterNodeParser(buffer_size=1, breakpoint_percentile_threshold=90, embed_model=embed_model)
    document = Document(text=text)
    nodes = parser.get_nodes_from_documents([document])
    return [node.text for node in nodes]

# Define the function for sentence window chunking
def sentence_window_chunk_text(text, embed_model):
    node_parser = SentenceWindowNodeParser.from_defaults(window_size=3, window_metadata_key="window", original_text_metadata_key="original_text")
    document = Document(text=text)
    nodes = node_parser.get_nodes_from_documents([document])
    return [node.text for node in nodes]  # Return list of strings

# Dictionary of chunking methods
chunking_methods = {
    "semantic_chunking": lambda text, embed_model: semantic_chunk_text(text, embed_model),
    "sentence_window_chunking": lambda text, embed_model: sentence_window_chunk_text(text, embed_model)
}

Then, I want to check different chunking methods in their ability to chunk my dataset using the same embedding method of "sentence-transformers/all-mpnet-base-v2"

In [120]:
# Apply the semantic chunking function to dataset

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")
Tesla_news['semantic_chunk_text'] = Tesla_news['clean_combined_text'].apply(lambda x: semantic_chunk_text(x, embed_model))
Tesla_news

,no.,date,source,title,link,content,combined_text,clean_combined_text,semantic_chunk_text
0,1,3-Jan-22,The New York Observer,"Tesla Crushed Every Expectation in 2021, Kicki...",https://www.proquest.com/usnews/docview/261613...,A Tesla Model X on display at the 3rd China In...,3-Jan-22 - Tesla Crushed Every Expectation in ...,3-Jan-22 - Tesla Crushed Every Expectation in ...,[3-Jan-22 - Tesla Crushed Every Expectation in...
1,2,8-Jan-22,Barron's (Online),Tesla's New Austin Factory to Start Up Soon. I...,https://www.proquest.com/usnews/docview/261780...,Christmas arrived a little late for Tesla bull...,8-Jan-22 - Tesla's New Austin Factory to Start...,8-Jan-22 - Tesla's New Austin Factory to Start...,[8-Jan-22 - Tesla's New Austin Factory to Star...
2,3,13-Jan-22,Wall Street Journal (Online),"Elon Musk, Hollywood Actors Criticize Californ...",https://www.proquest.com/usnews/docview/261928...,A proposal to cut incentives for rooftop solar...,"13-Jan-22 - Elon Musk, Hollywood Actors Critic...","13-Jan-22 - Elon Musk, Hollywood Actors Critic...","[13-Jan-22 - Elon Musk, Hollywood Actors Criti..."
3,4,14-Jan-22,Wall Street Journal (Online),Elon Musk-Backed EV Battery Technology Rises t...,https://www.proquest.com/usnews/docview/261953...,A less-expensive battery technology championed...,14-Jan-22 - Elon Musk-Backed EV Battery Techno...,14-Jan-22 - Elon Musk-Backed EV Battery Techno...,[14-Jan-22 - Elon Musk-Backed EV Battery Techn...
4,5,17-Jan-22,Republican & Herald,Smart progressives should love Elon Musk,https://www.proquest.com/usnews/docview/262085...,Nearly everything Sen. Elizabeth Warren tweete...,17-Jan-22 - Smart progressives should love Elo...,17-Jan-22 - Smart progressives should love Elo...,[17-Jan-22 - Smart progressives should love El...
...,...,...,...,...,...,...,...,...,...
545,546,30-Dec-22,"Wall Street Journal, Eastern edition",Biden's Back Door Tesla Subsidy,https://www.proquest.com/usnews/docview/275916...,Tesla's stock market value has plunged by some...,30-Dec-22 - Biden's Back Door Tesla Subsidy - ...,30-Dec-22 - Biden's Back Door Tesla Subsidy - ...,[30-Dec-22 - Biden's Back Door Tesla Subsidy -...
546,547,30-Dec-22,"New York Times, Late Edition (East Coast)","It Was a Bad Year for Stocks, and a Worse Year...",https://www.proquest.com/usnews/docview/275936...,FILE â Tesla vehicles charge at the One Pase...,"30-Dec-22 - It Was a Bad Year for Stocks, and ...","30-Dec-22 - It Was a Bad Year for Stocks, and ...","[30-Dec-22 - It Was a Bad Year for Stocks, and..."
547,548,30-Dec-22,"Wall Street Journal, Eastern edition","Musk, Wood Are Almost Right About The Fed",https://www.proquest.com/usnews/docview/275916...,[Financial Analysis and Commentary]\nSome of t...,"30-Dec-22 - Musk, Wood Are Almost Right About ...","30-Dec-22 - Musk, Wood Are Almost Right About ...","[30-Dec-22 - Musk, Wood Are Almost Right About..."
548,549,30-Dec-22,"New York Times, Late Edition (East Coast)",Looking Back on a Sobering Year in Silicon Val...,https://www.proquest.com/usnews/docview/275915...,A conversation with The Times's technology edi...,30-Dec-22 - Looking Back on a Sobering Year in...,30-Dec-22 - Looking Back on a Sobering Year in...,[30-Dec-22 - Looking Back on a Sobering Year i...


In [121]:
def print_chunks(dataframe, num_rows=5):
    for i in range(num_rows):
        print(f"Original Text {i+1}:\n{dataframe['clean_combined_text'].iloc[i]}\n")
        print("Chunks:")
        for j, chunk in enumerate(dataframe['semantic_chunk_text'].iloc[i]):
            print(f"Chunk {j+1}: {chunk}\n")  # Directly print the string
        print("="*80)

# Apply the semantic chunking function to the dataset
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Tesla_news['semantic_chunk_text'] = Tesla_news['clean_combined_text'].apply(lambda x: semantic_chunk_text(x, embed_model))

# Print chunks for the first 5 rows to see the results
print_chunks(Tesla_news, num_rows=5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Original Text 1:
3-Jan-22 - Tesla Crushed Every Expectation in 2021, Kicking off Elon Musk Roaring 2020 - A Tesla Model X on display at the 3rd China International Import Expo (CIIE) at the National Exhibition and Convention Center on November 5, 2020 in Shanghai, China. Guo Zhihua/VCG via Getty Images Despite Elon Musk s dramatic stock selloff that briefly put a dent in Tesla s share price in the final weeks of 2021, the company proved to investors that its core business remained solid. On Sunday, the electric carmaker reported record quarterly and full-year earnings that easily beat Wall Street expectations. Tesla delivered 308,000 vehicles in the fourth quarter and 936,000 vehicles in 2021, which almost doubled the total shipment of 2020. Analysts had expected the company to deliver around 270,000 vehicles in the fourth quarter and 897,000 vehicles for the full year. Tesla shares jumped more than 9 percent Monday morning on the news, pushing the company s market cap to $1.17 trillio

In [122]:
# Apply the semantic chunking function to dataset
Tesla_news['sentence_window_chunk_text'] = Tesla_news['clean_combined_text'].apply(lambda x: sentence_window_chunk_text(x, embed_model))
Tesla_news

,no.,date,source,title,link,content,combined_text,clean_combined_text,semantic_chunk_text,sentence_window_chunk_text
0,1,3-Jan-22,The New York Observer,"Tesla Crushed Every Expectation in 2021, Kicki...",https://www.proquest.com/usnews/docview/261613...,A Tesla Model X on display at the 3rd China In...,3-Jan-22 - Tesla Crushed Every Expectation in ...,3-Jan-22 - Tesla Crushed Every Expectation in ...,[3-Jan-22 - Tesla Crushed Every Expectation in...,[3-Jan-22 - Tesla Crushed Every Expectation in...
1,2,8-Jan-22,Barron's (Online),Tesla's New Austin Factory to Start Up Soon. I...,https://www.proquest.com/usnews/docview/261780...,Christmas arrived a little late for Tesla bull...,8-Jan-22 - Tesla's New Austin Factory to Start...,8-Jan-22 - Tesla's New Austin Factory to Start...,[8-Jan-22 - Tesla's New Austin Factory to Star...,[8-Jan-22 - Tesla's New Austin Factory to Star...
2,3,13-Jan-22,Wall Street Journal (Online),"Elon Musk, Hollywood Actors Criticize Californ...",https://www.proquest.com/usnews/docview/261928...,A proposal to cut incentives for rooftop solar...,"13-Jan-22 - Elon Musk, Hollywood Actors Critic...","13-Jan-22 - Elon Musk, Hollywood Actors Critic...","[13-Jan-22 - Elon Musk, Hollywood Actors Criti...","[13-Jan-22 - Elon Musk, Hollywood Actors Criti..."
3,4,14-Jan-22,Wall Street Journal (Online),Elon Musk-Backed EV Battery Technology Rises t...,https://www.proquest.com/usnews/docview/261953...,A less-expensive battery technology championed...,14-Jan-22 - Elon Musk-Backed EV Battery Techno...,14-Jan-22 - Elon Musk-Backed EV Battery Techno...,[14-Jan-22 - Elon Musk-Backed EV Battery Techn...,[14-Jan-22 - Elon Musk-Backed EV Battery Techn...
4,5,17-Jan-22,Republican & Herald,Smart progressives should love Elon Musk,https://www.proquest.com/usnews/docview/262085...,Nearly everything Sen. Elizabeth Warren tweete...,17-Jan-22 - Smart progressives should love Elo...,17-Jan-22 - Smart progressives should love Elo...,[17-Jan-22 - Smart progressives should love El...,[17-Jan-22 - Smart progressives should love El...
...,...,...,...,...,...,...,...,...,...,...
545,546,30-Dec-22,"Wall Street Journal, Eastern edition",Biden's Back Door Tesla Subsidy,https://www.proquest.com/usnews/docview/275916...,Tesla's stock market value has plunged by some...,30-Dec-22 - Biden's Back Door Tesla Subsidy - ...,30-Dec-22 - Biden's Back Door Tesla Subsidy - ...,[30-Dec-22 - Biden's Back Door Tesla Subsidy -...,[30-Dec-22 - Biden's Back Door Tesla Subsidy -...
546,547,30-Dec-22,"New York Times, Late Edition (East Coast)","It Was a Bad Year for Stocks, and a Worse Year...",https://www.proquest.com/usnews/docview/275936...,FILE â Tesla vehicles charge at the One Pase...,"30-Dec-22 - It Was a Bad Year for Stocks, and ...","30-Dec-22 - It Was a Bad Year for Stocks, and ...","[30-Dec-22 - It Was a Bad Year for Stocks, and...","[30-Dec-22 - It Was a Bad Year for Stocks, and..."
547,548,30-Dec-22,"Wall Street Journal, Eastern edition","Musk, Wood Are Almost Right About The Fed",https://www.proquest.com/usnews/docview/275916...,[Financial Analysis and Commentary]\nSome of t...,"30-Dec-22 - Musk, Wood Are Almost Right About ...","30-Dec-22 - Musk, Wood Are Almost Right About ...","[30-Dec-22 - Musk, Wood Are Almost Right About...","[30-Dec-22 - Musk, Wood Are Almost Right About..."
548,549,30-Dec-22,"New York Times, Late Edition (East Coast)",Looking Back on a Sobering Year in Silicon Val...,https://www.proquest.com/usnews/docview/275915...,A conversation with The Times's technology edi...,30-Dec-22 - Looking Back on a Sobering Year in...,30-Dec-22 - Looking Back on a Sobering Year in...,[30-Dec-22 - Looking Back on a Sobering Year i...,[30-Dec-22 - Looking Back on a Sobering Year i...


In [123]:
# Check some chunked texts after using semantic chunking
# Define a function to print the chunked text
def print_chunks(dataframe, num_rows=5):
    for i in range(num_rows):
        print(f"Original Text {i+1}:\n{dataframe['clean_combined_text'].iloc[i]}\n")
        print("Chunks:")
        for j, chunk in enumerate(dataframe['sentence_window_chunk_text'].iloc[i]):
            print(f"Chunk {j+1}: {chunk}\n")  # Directly print the string
        print("="*80)

# Print chunks for the first 5 rows to see the results
print_chunks(Tesla_news, num_rows=5)

Original Text 1:
3-Jan-22 - Tesla Crushed Every Expectation in 2021, Kicking off Elon Musk Roaring 2020 - A Tesla Model X on display at the 3rd China International Import Expo (CIIE) at the National Exhibition and Convention Center on November 5, 2020 in Shanghai, China. Guo Zhihua/VCG via Getty Images Despite Elon Musk s dramatic stock selloff that briefly put a dent in Tesla s share price in the final weeks of 2021, the company proved to investors that its core business remained solid. On Sunday, the electric carmaker reported record quarterly and full-year earnings that easily beat Wall Street expectations. Tesla delivered 308,000 vehicles in the fourth quarter and 936,000 vehicles in 2021, which almost doubled the total shipment of 2020. Analysts had expected the company to deliver around 270,000 vehicles in the fourth quarter and 897,000 vehicles for the full year. Tesla shares jumped more than 9 percent Monday morning on the news, pushing the company s market cap to $1.17 trillio

We can see that sentence window chunking slipt my dataset into more chunk than semantic chunking

## 3.2 Flatten chunking

In [124]:
# Flatten chunks to a DataFrame
def flatten_chunks(dataframe):
    chunk_data = []
    for _, row in dataframe.iterrows():
        for chunk in row['chunk_text']:
            chunk_data.append({
                'date': row['date'],
                'source': row['source'],
                'title': row['title'],
                'link': row['link'],
                'chunk_text': chunk,  # Use chunk directly as it's a string
                'embedding': embed_model.get_text_embedding(chunk)  # Directly use the embedding
            })
    return pd.DataFrame(chunk_data)

## 3.3 Setup a vector database

Now we will setup a vector database (this time using Chroma DB) and index our documents (via embeddings).

In [125]:
# Set up ChromaDB client and collection
client = chromadb.PersistentClient(path="./chroma_db")

# Check if the collection exists and delete it if it does
if "Tesla_news_collection" in client.list_collections():
    client.delete_collection("Tesla_news_collection")

In [130]:
# Create a new collection
chroma_collection = client.create_collection("Tesla_News_Collections")


In [131]:
# Function to setup the vector store using ChromaDB
def setup_vector_store(chunk_df, embed_model, chunking_name):
    collection_name = f"Tesla_news_{chunking_name}_collection"

    # Set up ChromaDB persistent client and collection
    client = chromadb.PersistentClient(path="./chroma_db")

    # Check if the collection exists and delete it if it does
    if collection_name in [c.name for c in client.list_collections()]:
        client.delete_collection(collection_name)

    # Create a new collection
    chroma_collection = client.create_collection(collection_name)

    # Ensure embedding dimensions are consistent
    first_embedding_dim = len(embed_model.get_text_embedding(chunk_df['chunk_text'].iloc[0]))

    for idx, row in chunk_df.iterrows():
        embedding = embed_model.get_text_embedding(row['chunk_text'])
        chroma_collection.add(
            ids=[str(idx)],
            documents=[row['chunk_text']],
            metadatas=[{
                'index': str(idx),
                'date': row['date'],
                'source': row['source'],
                'title': row['title'],
                'link': row['link']
            }],
            embeddings=[embedding]  # Directly use embedding here
        )
    return chroma_collection

In [132]:
# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

name='Tesla_News_Collections' id=UUID('2187e940-e451-4642-a8e8-009d2f863da0') metadata=None tenant='default_tenant' database='default_database'
Collection name is: Tesla_News_Collections


## 3.4 Query Expansion

In [133]:
# Function to expand query using an open-source model
generator = pipeline('text-generation', model='gpt2')

In [134]:
# Function to expand query using an open-source model
def expand_query(query):
    prompt = f"Expand the following query with additional relevant terms:\n{query}"
    response = generator(prompt, num_return_sequences=1, truncation=True)
    expanded_query = response[0]['generated_text'].strip()
    return expanded_query

## 3.5 Hybrid Search

In [135]:
# Hybrid search function
def hybrid_search(query, tfidf_vectorizer, tfidf_matrix, chunk_df, embed_model, chroma_collection, k=10):
    # Keyword-based search using TF-IDF
    query_tfidf = tfidf_vectorizer.transform([query])
    tfidf_scores = np.dot(tfidf_matrix.toarray(), query_tfidf.T.toarray()).flatten()
    tfidf_top_k_idx = tfidf_scores.argsort()[-k:][::-1]
    keyword_retrieved_docs = chunk_df.iloc[tfidf_top_k_idx]

    # Semantic search using vector embeddings
    query_embedding = embed_model.get_text_embedding(query)
    results = chroma_collection.query(query_embeddings=[query_embedding], n_results=k)

    # Extract the DataFrame indices from the metadata and validate them
    retrieved_indices = [doc['index'] for doc in results['metadatas'][0] if 'index' in doc and int(doc['index']) in chunk_df.index]
    valid_retrieved_indices = [int(idx) for idx in retrieved_indices if int(idx) in chunk_df.index]
    semantic_retrieved_docs = chunk_df.loc[valid_retrieved_indices]

    # Combine results from both searches & Drop duplicates based on columns that do not contain lists
    combined_docs = pd.concat([keyword_retrieved_docs, semantic_retrieved_docs]).drop_duplicates(subset=['chunk_text', 'date', 'source', 'title', 'link'])
    return combined_docs


## 3.6 Re-ranking

In [136]:
# Re-ranking function
def re_rank_documents(query, retrieved_docs, embed_model):
    query_embedding = np.array(embed_model.get_text_embedding(query)).reshape(1, -1)
    retrieved_embeddings = np.vstack(retrieved_docs['embedding'].apply(lambda x: np.array(x).flatten()).tolist())

    # Ensure the dimensions match for dot product
    if len(query_embedding.shape) != 2 or len(retrieved_embeddings.shape) != 2:
        raise ValueError(f"Embeddings must be 2D arrays. Got query_embedding with shape {query_embedding.shape} and retrieved_embeddings with shape {retrieved_embeddings.shape}")

    # Ensure the dimensions match for dot product
    if query_embedding.shape[1] != retrieved_embeddings.shape[1]:
        raise ValueError(f"Shape mismatch: query_embedding has shape {query_embedding.shape} and retrieved_embeddings has shape {retrieved_embeddings.shape}")

    scores = np.dot(retrieved_embeddings, query_embedding.T).flatten()
    retrieved_docs['score'] = scores
    ranked_docs = retrieved_docs.sort_values(by='score', ascending=False)
    return ranked_docs

## 3.7 Prompt Template

In [137]:
# Enhanced prompt template for the financial news domain
qa_prompt_str = (
    "You are an expert financial analyst with a deep understanding of market trends, Elon Musk and Tesla's business, especially in 2022. "
    "You have access to the following financial news articles about Tesla and Elon Musk in 2022:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Based on the provided information, please answer the following question: {query_str}\n"
    "Ensure your response is concise, informative, and well-reasoned, citing relevant dates, sources, financial metrics, and key points from the articles. "
    "Consider the impact of specific events, market trends, and financial outcomes on Tesla and Elon Musk in 2022."
)

In [138]:
# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "You are an expert financial analyst. "
            "Always answer the question based on the provided context, "
            "even if the context isn't directly helpful. "
            "Your responses should highlight key financial metrics, events, and their impacts on Tesla and Elon Musk."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

## 3.8 Develope Query pipeline for Retrieval augmentation

In [139]:
# Function to query the pipeline using the LLama model
def query_pipeline(user_query, tfidf_vectorizer, tfidf_matrix, chunk_df, embed_model, chroma_collection, text_qa_template, index, mode):
    # Step 1: HyDE Query Transformation
    hyde = HyDEQueryTransform(include_original=True)
    transformed_query = TransformQueryEngine(user_query, query_transform=hyde)

    # Step 2: Expand the transformed query
    expanded_query = expand_query(transformed_query)

    # Step 3: Hybrid search with expanded transformed query
    retrieved_docs = hybrid_search(expanded_query, tfidf_vectorizer, tfidf_matrix, chunk_df, embed_model, chroma_collection)

    # Step 4: Re-rank the retrieved documents
    ranked_docs = re_rank_documents(expanded_query, retrieved_docs, embed_model)

    # Step 5: Format the context
    context = '\n'.join(ranked_docs['chunk_text'].head(5))  # Select top 5 re-ranked documents

    # Step 6: Use the vector index and query engine with HyDE transformation to get the response from LLM
    query_engine_with_hyde = index.as_query_engine(text_qa_template=text_qa_template, llm=llm, response_mode=mode)
    response = query_engine_with_hyde.query(f"{user_query}")

    return response.response

# Part 4: Evaluation

In [140]:
# Function to extract numbers from text as a set of strings
def extract_numbers(text):
    return set(re.findall(r'\d+%?|\d+\.\d+%?', text))

# Function to extract important words from text as a set of strings
def extract_words(text):
    # Capture important words (excluding common stopwords)
    words = set(re.findall(r'\b\w+\b', text.lower())) - {'the', 'and', 'a', 'to', 'in', 'of', 'for', 'on', 'is', 'at', 'with', 'as', 'by', 'it', 'that', 'from', 'this', 'an', 'be', 'which', 'or', 'are', 'but', 'not'}
    return words

In [145]:
def test_responses(queries, expected_answers, embedding_models, chunking_methods, text_qa_template):
    # Initialize a dictionary to keep track of correct responses for each combination of embedding model, chunking method, and mode
    correct = {embed: {chunk: {"compact": 0, "refine": 0, "tree_summarize": 0} for chunk in chunking_methods} for embed in embedding_models}
    total = len(queries)  # Total number of queries
    modes = ["compact", "refine", "tree_summarize"]  # Different modes of response generation

    # Iterate through each embedding model
    for embed_model_name, embed_model in embedding_models.items():
        Settings.embed_model = embed_model  # Set the current embedding model in settings

        # Iterate through each chunking method
        for chunking_name, chunk_method in chunking_methods.items():
            # Apply chunking method to clean combined text
            Tesla_news['chunk_text'] = Tesla_news['clean_combined_text'].apply(lambda text: chunk_method(text, embed_model))
            chunk_df = flatten_chunks(Tesla_news)  # Flatten the chunks into a DataFrame
            tfidf_vectorizer = TfidfVectorizer()  # Initialize TF-IDF vectorizer
            tfidf_matrix = tfidf_vectorizer.fit_transform(chunk_df['chunk_text'])  # Fit and transform chunk text with TF-IDF

            # Get embeddings for chunk text
            chunk_df['embedding'] = chunk_df['chunk_text'].apply(lambda text: embed_model.get_text_embedding(text))
            chroma_collection = setup_vector_store(chunk_df, embed_model, chunking_name)  # Setup vector store
            vector_store = ChromaVectorStore(chroma_collection=chroma_collection)  # Create Chroma vector store
            storage_context = StorageContext.from_defaults(vector_store=vector_store)  # Initialize storage context with vector store

            # Create an index from documents
            index = VectorStoreIndex.from_documents([Document(text=row['chunk_text']) for _, row in chunk_df.iterrows()], storage_context=storage_context, embed_model=embed_model)

            # Iterate through each query and its expected answer
            for query, expected in zip(queries, expected_answers):
                print(f"Query: {query}")
                expected_numbers = extract_numbers(expected)  # Extract numbers from the expected answer
                expected_words = extract_words(expected)  # Extract words from the expected answer

                # Iterate through each mode of response generation
                for mode in modes:
                    try:
                        # Generate response using query pipeline
                        response = query_pipeline(query, tfidf_vectorizer, tfidf_matrix, chunk_df, embed_model, chroma_collection, text_qa_template, index, mode)
                        response_numbers = extract_numbers(response)  # Extract numbers from the response
                        response_words = extract_words(response)  # Extract words from the response

                        # Check keyword matches
                        common_numbers = expected_numbers.intersection(response_numbers)
                        common_words = expected_words.intersection(response_words)

                        # Determine correctness based on the presence of expected numbers or words
                        if expected_numbers:
                            is_correct = len(common_numbers) > 0
                        else:
                            is_correct = len(common_words) > 0

                        # Increment the correct count for the current combination
                        correct[embed_model_name][chunking_name][mode] += int(is_correct)
                        print(f"Embedding Model: {embed_model_name}, Chunking Method: {chunking_name}, Response Mode: {mode}")
                        print(f"Response: {response}")
                        print(f"Expected Answer: {expected}")
                        print(f"Common Numbers: {common_numbers}")
                        print(f"Common Words: {common_words}")
                        print(f"Correct: {is_correct}")
                        print("="*80)
                    except Exception as e:
                        # Handle and print errors
                        print(f"Embedding Model: {embed_model_name}, Chunking Method: {chunking_name}, Response Mode: {mode}")
                        print(f"Error: {str(e)}")
                        print("="*80)

    # Uncomment the following block to print the accuracy of each combination
    # for embed_model_name, chunking_results in correct.items():
    #     for chunk_method_name, mode_results in chunking_results.items():
    #         for mode, count in mode_results.items():
    #             # Calculate the accuracy for the current combination
    #             accuracy = count / total
    #             # Print the accuracy percentage
    #             print(f"Accuracy for {embed_model_name} with {chunk_method_name} in mode {mode}: {accuracy * 100:.2f}%")

In [ ]:
# Define test queries and expected answers
test_queries_1 = ["How much did Twitter's stock surge after Elon Musk disclosed his stake in the company? "]
expected_answers_1 = ["Twitter's stock surged 27%."]

# Run the evaluation
results_1 = test_responses(test_queries_1, expected_answers_1, embedding_models, chunking_methods, text_qa_template)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: How much did Twitter's stock surge after Elon Musk disclosed his stake in the company? 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: BAAI/bge-small-en-v1.5, Chunking Method: semantic_chunking, Response Mode: compact
Response:  According to the article "Elon Musk' s Twitter Tantrum Needs a Time Out; Twitter shares soared Monday after Musk's stake disclosure, but they will soon find gravity" published by Wall Street Journal on April 4th, 2022, Twitter's stock surged 27% Monday after Elon Musk disclosed his 9% stake in the company worth about $3 billion. This makes Mr. Musk the largest shareholder of Twitter and investors may pay for his interest. The article highlights that Mr. Musk is paying to change the conversation on Twitter and it remains to be seen how this will impact the social media platform's future growth and profitability.

Expected Answer: Twitter's stock surged 27%.
Common Numbers: {'27%'}
Common Words: {'surged', 'twitter', 'stock', 's', '27'}
Correct: True


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Query: How much did Twitter's stock surge after Elon Musk disclosed his stake in the company? 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: compact
Response:  According to the provided information, Twitter's stock surged after Elon Musk disclosed his stake in the company. The article notes that Musk disclosed a roughly 9% stake in early April, which led to a 32% increase in Twitter shares. Additionally, the article mentions that Musk took Twitter private in an $44 billion deal, marking the close of a dramatic courtship between the two companies. Based on these events and trends, it can be inferred that Elon Musk's involvement with Tesla and other businesses has had a significant impact on their financial outcomes in 2022.

Expected Answer: Twitter's stock surged 27%.
Common Numbers: set()
Common Words: {'stock', 's', 'surged', 'twitter'}
Correct: False


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: refine
Response:  The answer you provided is correct. According to Barron's (Online) article dated 12-Apr-22, Elon Musk disclosed his roughly 9% stake in Twitter which caused the company's stock to surge by about 20%. This news has highlighted the key-man risk for Tesla investors and could potentially affect Tesla's performance if it impacts their relationship with Musk or his role in the company. Additionally, this event can be attributed to Twitter's increased user engagement and advertising revenue due to Musk's influence on the platform.

Expected Answer: Twitter's stock surged 27%.
Common Numbers: set()
Common Words: {'stock', 's', 'twitter'}
Correct: False
Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: tree_summarize
Response:  The given context mentions that Elon Musk disclosed his 9% stake in the

In [33]:
# Define test queries and expected answers
test_queries_2 = ["How much does Tesla's Full Self-Driving package cost after the recent price increase? "]
expected_answers_2 = ["Full Self-Driving is a suite of advanced driver-assistance features that Tesla sells for $12000."]

# Run the evaluation
results_2 = test_responses(test_queries_2, expected_answers_2, embedding_models, chunking_methods, text_qa_template)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: How much does Tesla's Full Self-Driving package cost after the recent price increase? 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: BAAI/bge-small-en-v1.5, Chunking Method: semantic_chunking, Response Mode: compact
Response:  According to the article by Rebecca Elliott published on October 20, 2022, the price of Tesla's Full Self-Driving package has increased to $15,000. This represents a significant increase from its original price of $10,000, indicating that the demand for this feature is high and Tesla is able to charge more for it. The article also states that Elon Musk expects to broaden access to this advanced driver-assistance system by the end of the year, but does not provide any further details on when or how this will happen. Overall, these developments suggest that Tesla's business continues to thrive in 202

Expected Answer: Full Self-Driving is a suite of advanced driver-assistance features that Tesla sells for $12000.
Common Numbers: set()
Common Words: {'driver', 'advanced', 'tesla', 'self', 'full', 'driving', 'assistance'}
Correct: False


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Query: How much does Tesla's Full Self-Driving package cost after the recent price increase? 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: compact
Response:  According to the Wall Street Journal article "Elon Musk' Decision to Slow New Tesla Models Risks Holding Up Growth; CEO says company doesn't plan to introduce any new vehicle models in 2022," which is dated 28 January 2022, there is no information available about a recent price increase for Tesla's Full Self-Driving package. However, as of 2021, the cost of Tesla's Full Self-Driving package was $12,000.

Expected Answer: Full Self-Driving is a suite of advanced driver-assistance features that Tesla sells for $12000.
Common Numbers: set()
Common Words: {'full', 'tesla', 'driving', 'self'}
Correct: False


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: refine
Response:  As of now, the cost of Tesla's Full Self-Driving package is $12,000. However, with recent price increases and new product releases, it can be challenging to determine the exact pricing for this feature at this time. It's essential to keep an eye on any updates or announcements from Tesla regarding this matter.

Expected Answer: Full Self-Driving is a suite of advanced driver-assistance features that Tesla sells for $12000.
Common Numbers: set()
Common Words: {'full', 'tesla', 'driving', 'self'}
Correct: False
Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: tree_summarize
Response:  According to multiple sources, Tesla recently increased the price of their Full Self-Driving package by 20% to $12,000. However, it is important to note that this information may not be entirely accurate and s

In [45]:
# Define test queries and expected answers
test_queries_3 = ["How did the shares of larger lithium mining companies, Albemarle and SQM, react to Musk's tweet lamenting the rising cost of lithium? "]
expected_answers_3 = ["Albemarle shares were down about 1.2%, and SQM shares were down about 2.8%."]

# Run the evaluation
results_3 = test_responses(test_queries_3, expected_answers_3, embedding_models, chunking_methods, text_qa_template)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: How did the shares of larger lithium mining companies, Albemarle and SQM, react to Musk's tweet lamenting the rising cost of lithium? 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: BAAI/bge-small-en-v1.5, Chunking Method: semantic_chunking, Response Mode: compact
Response:  The shares of larger lithium mining companies such as Albemarle and SQM were negatively impacted by Musk' tweet lamenting the rising cost of lithium. Musk's tweet brought attention to the increasing costs of materials needed for electric vehicles, which could potentially lead to lower demand for these metals in the long run. This increase in cost can cause concern for investors, resulting in a sell-off in their stocks. Additionally, Tesla's financial performance is closely tied to the price of lithium, and any negative sentiment towards the metal could further impact the company's stock price.

Expected Answer: Albemarle shares were down about 1.2%, and SQM shares were down about 2.8%.
Common Numbers: set()
Common Words: {'were', 'albemarle', 'sqm', 'shares'}
Correct: False


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Query: How did the shares of larger lithium mining companies, Albemarle and SQM, react to Musk's tweet lamenting the rising cost of lithium? 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: compact
Response:  In July 2022, Tesla's shares declined due to market uncertainty caused by Elon Musk's tweet lamenting the rising cost of lithium. The plummeting price of Tesla Inc. occurred despite its $44 billion takeover deal with Twitter. Additionally, larger lithium mining companies such as Albemarle and SQM were not pleased with Musk's comments on the matter. 

The Wall Street Journal reported that Tesla's shares had plummeted in July 2022 due to market uncertainty caused by Elon Musk's tweet. The article highlighted that the $44 billion takeover deal between Tesla and Twitter was less appealing over time as a result of plunging shares in Tesla and a darkening outlook for social-media companies.

In addition, larger lithium mining companies such as Albemarle and SQM were not pleased with Musk's comments on the matter. This is because Musk had expressed his concern

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: refine
Response:  If the stock price increase occurred before Musk' tweets about rising costs, then the cost of mining from Company B would still be less than companies A and C as it was in the original question. In this case, it is more cost-effective for Tesla to work with Company B. This is because while the initial impact of the stock price increase makes Company B a cheaper option, the subsequent decrease in costs due to Musk' tweet reduces the advantage that Company B had over companies A and C. 
Therefore, the company should be recommended based on whether or not Elon Musk’s tweets about rising costs have occurred before or after the stock price increase.

Expected Answer: Albemarle shares were down about 1.2%, and SQM shares were down about 2.8%.
Common Numbers: set()
Common Words: {'about'}
Correct: False
Embedding Model: sentence-transformers/all-mpnet-base-v2, 

In [67]:
# Define test queries and expected answers
test_queries_4 = ["What were the claims of the California Department of Fair Employment and Housing against Tesla? "]
expected_answers_4 = ["The California Department of Fair Employment and Housing claimed that employees were subject to racial slurs, assigned physically arduous work, and denied transfers and promotions."]

# Run the evaluation
results_4 = test_responses(test_queries_4, expected_answers_4, embedding_models, chunking_methods, text_qa_template)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What were the claims of the California Department of Fair Employment and Housing against Tesla? 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: BAAI/bge-small-en-v1.5, Chunking Method: semantic_chunking, Response Mode: compact
Response:  According to the Wall Street Journal article (2022), the California Department of Fair Employment and Housing filed a lawsuit against Tesla in February 2022. The lawsuit alleges that Black workers at Tesla's Fremont, Calif., assembly plant are subjected to racial discrimination and harassment by supervisors. Specifically, the article mentions that Black workers heard racial slurs being used by supervisors and were confronted with racist graffiti in the factory. These claims of workplace discrimination have resulted in scrutiny from both state and federal employment authorities. It is important for Tesla to address these issues as they can negatively impact employee morale, retention, and overall company culture, which could ultimately affect their financial performance.

Expected Answer: The California Department of Fair Employment and Housing claimed that employees were subje

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Query: What were the claims of the California Department of Fair Employment and Housing against Tesla? 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: compact
Response:  The California Department of Fair Employment and Housing (DFEH) filed a lawsuit against Tesla for discrimination, harassment, retaliation, and mistreatment of employees in the workplace. The DFEH received numerous complaints from Tesla's employees, including instances of racial slurs, physically demanding work, and denied promotions and transfers. Tesla responded by stating that they do not tolerate any form of discrimination or mistreatment of their employees, as stated in the provided articles. These claims highlight the importance of maintaining a safe and inclusive work environment for all employees, as well as the potential financial impact on Tesla's reputation and bottom line if such issues are not addressed promptly and effectively.

Expected Answer: The California Department of Fair Employment and Housing claimed that employees were subject to 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: refine
Response:  Based on the new context provided by the user, the California Department of Fair Employment and Housing (DFEH) sued Tesla in February 2022 after receiving hundreds of complaints from employees who said they were subject to racial slurs, assigned physically arduous work, denied transfers and promotions. The resolution was for $31 million settlement.
Tesla's response is that they do not tolerate discrimination, harassment, retaliation or any mistreatment of employees in the workplace or work-related situations. 
This lawsuit highlights the importance of creating an inclusive workplace culture that values diversity, equity, and inclusion, as well as complying with employment laws and regulations. The impact on Tesla's financial performance is unclear, but the settlement could have implications for future shareholder value and investor confidence in the comp

In [84]:
# Define test queries and expected answers
test_queries_5 = ["Discuss the broader implications of the Federal Reserve's rate hikes on the tech sector as articulated by both Elon Musk and Cathie Wood. "]
expected_answers_5 = ["Both Elon Musk and Cathie Wood have articulated that the Federal Reserve's rate hikes have negatively impacted the tech sector by making risk-free Treasurys more attractive, thus reducing the value of more-risky equity assets. This has contributed to significant declines in tech stock valuations. While they point to rising interest rates as a major factor, other macroeconomic conditions such as inflation, supply chain disruptions, and changing consumer behavior also play critical roles. The broader implication is that tech companies must navigate a complex economic landscape where multiple factors influence their performance and investor sentiment."]

# Run the evaluation
results_5 = test_responses(test_queries_5, expected_answers_5, embedding_models, chunking_methods, text_qa_template)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: Discuss the broader implications of the Federal Reserve's rate hikes on the tech sector as articulated by both Elon Musk and Cathie Wood. 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: BAAI/bge-small-en-v1.5, Chunking Method: semantic_chunking, Response Mode: compact
Response:  As an expert financial analyst with a deep understanding of Tesla and Elon Musk's business operations, I can say that the Federal Reserve'S rate hikes have had significant implications for the tech sector as articulated by Elon Musk and Cathie Wood. Specifically, higher interest rates are causing consumers to spend less on technology products and services, leading to decreased revenue for tech companies like Tesla. 

Elon Musk's comments in a recent interview suggest that he believes high-interest rates will have a negative impact on Tesla's ability to secure financing for its electric vehicles and other projects. Additionally, Musk has expressed concerns about the potential for rising inflation to erode the value of Tesla's stock. 

Cathie Wood, on the other hand, sees higher interest rates as an opportunity for investors to buy tech stocks at a discount. She argues that the 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Query: Discuss the broader implications of the Federal Reserve's rate hikes on the tech sector as articulated by both Elon Musk and Cathie Wood. 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: compact
Response:  The Federal Reserve's rate hikes have implications for the tech sector as articulated by both Elon Musk and Cathie Wood. According to Barron'S' "Tech Earnings Are a Reflection of the Postpandemic Economy. Facebook's Parent Is Next." article published on 27-Apr-22, the Fed's decision to raise interest rates has led to increased inflation and a higher cost of borrowing for businesses in the tech sector. This can impact their ability to invest in research and development, as well as expand operations. Elon Musk has also expressed concerns about inflation and its potential impact on Tesla's business.

In contrast, economist at Berenberg Capital, who discussed the Fed's actions and inflation control measures, suggests that rising interest rates could have a positive impact on the stock market in general, including the tech sector. This is because higher inte

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: sentence-transformers/all-mpnet-base-v2, Chunking Method: sentence_window_chunking, Response Mode: refine
Response:  The impact of the Federal Reserve' rate hikes on the tech industry as discussed by Elon Musk and Cathie Wood. 
As an AI assistant, I have provided you with a new answer based on the context you provided. Please note that this is just one possible approach to answering your question and there may be multiple other valid answers.
Expected Answer: Both Elon Musk and Cathie Wood have articulated that the Federal Reserve's rate hikes have negatively impacted the tech sector by making risk-free Treasurys more attractive, thus reducing the value of more-risky equity assets. This has contributed to significant declines in tech stock valuations. While they point to rising interest rates as a major factor, other macroeconomic conditions such as inflation, supply chain disruptions, and changing consumer behavior also play critical roles. The broader implication is 

To ensure the RAG system's capability to generate content based on knowledge not publicly available at the point of training, additional queries were tested that the contents are not included in the dataset.

In [ ]:
# Define test queries and expected answers
test_queries_6 = ["What is the lastest news about Apple in 2022?"]
expected_answers_6 = [""]

# Run the evaluation
results_6 = test_responses(test_queries_6, expected_answers_6, embedding_models, chunking_methods , text_qa_template)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What is the lastest news about Apple in 2022?


INFO:backoff:Backing off send_request(...) for 0.1s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: BAAI/bge-small-en-v1.5, Chunking Method: semantic_chunking, Response Mode: compact
Response:  According to the provided news articles, Apple has recently frozen prices for its iPhone range, a move that is designed to keep consumers spending even during a cost-of-living crisis. The decision to freeze prices suggests that Apple is taking greater control of its supply chain and microchip design, which could be seen as a positive development for Tesla and Elon Musk in 2022. However, it remains to be seen how this decision will impact the company's sales or profitability in the long term. Additionally, concerns about a potential recession may also impact Tesla and Musk's business prospects in the coming years.

Expected Answer: 
Common Numbers: set()
Common Words: set()
Correct: False


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Embedding Model: BAAI/bge-small-en-v1.5, Chunking Method: semantic_chunking, Response Mode: refine
Response:  The lastest news about Apple in  20222 is that they have frozen prices for their latest product launch, including the iPhone 14 range. This move suggests that Apple is taking steps to protect its $200 billion franchise by preventing customers from upgrading handsets due to a cost-of-living crisis while inflation ramps up costs. Although this may seem like a positive gesture for consumers, it could also signal vulnerability in the long run as the company faces potential challenges in maintaining profitability and competing with other tech giants such as Samsung.

Expected Answer: 
Common Numbers: set()
Common Words: set()
Correct: False
Embedding Model: BAAI/bge-small-en-v1.5, Chunking Method: semantic_chunking, Response Mode: tree_summarize
Response:  According to the context information provided, the latest news about Apple in 2022 is that it has frozen prices for its iPhone 1

In [92]:
# Define test queries and expected answers
test_queries_7 = ["What is the current employment situation in the UK in 2024?"]
expected_answers_7 = [""]

# Run the evaluation
results_7 = test_responses(test_queries_7, expected_answers_7, embedding_models, chunking_methods, text_qa_template)


AttributeError: 'str' object has no attribute 'items'